In [ ]:
import numpy as np

In [ ]:
from sdf.netowrk import SqueezeSDFNetwork as SDFNetwork 

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [ ]:
model = SDFNetwork(encoding="hashgrid")

In [ ]:
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
import glob

In [ ]:
def scale_fn():
    return 1

checkpoint_list = sorted(glob.glob(f'trial_sdf/checkpoints/ngp_ep0161.pth.tar'))
if checkpoint_list:
    checkpoint = checkpoint_list[-1]
checkpoint_dict = torch.load(checkpoint, map_location=device)

In [ ]:
missing_keys, unexpected_keys = model.load_state_dict(checkpoint_dict['model'], strict=False)
missing_keys, unexpected_keys

In [ ]:
bound = 1
N = 512
x_ = np.linspace(-bound, bound, N)
y_ = np.linspace(-bound, bound, N)
z_ = np.linspace(-bound, bound, N)
x, y = np.meshgrid(x_, y_, indexing='ij')
XY = np.stack([x, y], axis=-1).reshape((-1, 2))
x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')
X = np.stack([x, y, z], axis=-1).reshape((-1, 3))

In [ ]:
from tqdm.auto import tqdm
model = model.eval()
model = model.to(device)
batch = 2**16
encoders = [model.encoder_xy, model.encoder_yz, model.encoder_zx]
backbones = [model.backbone_xy, model.backbone_yz, model.backbone_zx]
names = ["xy", "yz", "zx"]
for encoder, backbone, name in zip(encoders, backbones, names):
    results = []
    with torch.cuda.amp.autocast(enabled=True):
        for k in tqdm(range(int(XY.shape[0] / batch))):
            X = torch.tensor(XY[k * batch:(k + 1) * batch].astype(np.float32)).to(device)
            h = model.forward_backbone(X, backbone, encoder)
            results.append(h.detach().cpu().numpy())
    results_ = np.vstack(results).reshape([N, N, -1])    
    with open(f"cache_{name}_32_p1.npy", "wb") as f:
        np.save(f, results_[:, :, :4].astype(np.float32), allow_pickle=False)
    with open(f"cache_{name}_32_p2.npy", "wb") as f:
        np.save(f, results_[:, :, 4:].astype(np.float32), allow_pickle=False)

In [ ]:
from tqdm.auto import tqdm
model = model.eval()
model = model.to(device)
batch = 2**16
results = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(X.shape[0] / batch))):
        x = torch.tensor(X[k * batch:(k + 1) * batch].astype(np.float32)).to(device)
        xy = x[:, [0, 1]]
        yz = x[:, [1, 2]]
        zx = x[:, [2, 0]]

        h_xy = model.forward_backbone(xy, model.backbone_xy, model.encoder_xy)
        h_yz = model.forward_backbone(yz, model.backbone_yz, model.encoder_yz)
        h_zx = model.forward_backbone(zx, model.backbone_zx, model.encoder_zx)

        h1 = (h_xy * h_yz).sum(1)[:, None]
        h2 = (h_yz * h_zx).sum(1)[:, None]
        h3 = (h_zx * h_xy).sum(1)[:, None]


        h = torch.cat([h1, h2, h3], dim=1)

        h = model.head(h)
        results.append(h.detach().cpu().numpy())
    results_ = np.vstack(results).reshape([N, N, -1])    

In [ ]:
results_.shape

In [ ]:
results_.max()

In [ ]:
results_scaled =np.clip(np.interp(
        results_, (0, results_.max()), (0, 255)).astype("uint8"), 0, 255)

In [ ]:
from PIL import Image
for i in tqdm(range(512)):
    a = results_scaled[:, i, :]
    img = Image.fromarray(np.stack([a, a, a], axis=2), 'RGB')
    img.save(f'cache_sdf\img{i}.png')
    print(i)

In [ ]:
next(model.head.parameters())

In [ ]:
from PIL import Image
import numpy as np

# w, h = 512, 512
data = results_[:, :, :3]
# data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(np.interp(data, (data.min(), data.max()),(0, 255)).astype("uint8"), 'RGB')
# img.save('my.png')
img.show()

In [ ]:
X.shape

In [ ]:
import math

def spherical_to_cartesian(r, theta, phi):
    x = r * math.sin(theta) * math.cos(phi)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)
    return (x, y, z)

In [ ]:
bound = opt.bound
N = 2048
theta_ = np.linspace(0, 1 , N) ** 1 * np.pi
phi_ = np.linspace(0, 1, N) ** 1 * 2 * np.pi
theta, phi = np.meshgrid(theta_, phi_, indexing='ij')
x = np.sin(theta) * np.cos(phi);
z = np.sin(theta) * np.sin(phi);
y = np.cos(theta);
D = np.stack([x, y, z], axis=-1).reshape((-1, 3))

In [ ]:
bound = opt.bound
N = 2048
theta_ = np.linspace(0, 1 , N) ** 1 * np.pi
phi_ = np.linspace(0, 1, N) ** 1 * 2 * np.pi
theta, phi = np.meshgrid(theta_, phi_, indexing='ij')



x = np.sin(theta) * np.cos(phi);
y = np.sin(theta) * np.sin(phi);
z = np.cos(theta);
D = np.stack([x, y, z], axis=-1).reshape((-1, 3))

In [ ]:
# xyz -> yzx
theta = torch.acos(d[:, 1])
phi = torch.atan2(d[:, 2], d[:, 0])

In [ ]:
D.shape

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Helix equation
t = np.linspace(0, 10, 50)
# x, y, z = np.cos(t), np.sin(t), t

fig = go.Figure(data=[go.Scatter3d(x=x.flatten(), y=y.flatten(), z=z.flatten(),
                                   mode='markers',
                                   marker=dict(size=0.2,)
                                  )])
fig.show()

In [ ]:
model = model.eval()

In [ ]:
model = model.to(device)

In [ ]:
torch.tensor(X[:10].astype(np.float16)).to(device).dtype

In [ ]:
batch = 2**16

In [ ]:
from tqdm.auto import tqdm

In [ ]:
results = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(X.shape[0] / batch))):
        x = torch.tensor(X[k * batch:(k + 1) * batch].astype(np.float32)).to(device)
        x_ = torch.floor(x / (2 * model.bound / model.resolution)) + model.resolution // 2
        x_ = torch.clamp(x_, 0, model.resolution - 1)
        c = (x_ - model.resolution // 2 + 0.5) * (2 * model.bound / model.resolution)

        x_ = x_[:, 0] * model.resolution * model.resolution + x_[:, 1] * model.resolution + x_[:, 2]
        bids = x_.int()
        result = torch.zeros((x.shape[0], 8)).half()
        for bid in torch.unique(bids):
            pos = x[x_ == bid]
            centers = c[x_ == bid]
            h = model.encoders[bid](pos - centers, bound=model.bound / model.resolution)
            for l in model.sigma_nets[bid]:
                h = l(h)
            result[x_ == bid] = h.detach().cpu()      
        results.append(result.numpy())

In [ ]:
results = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(X.shape[0] / batch))):
        h = model.encoder(torch.tensor(X[k * batch:(k + 1) * batch].astype(np.float32)).to(device))
        for l in model.sigma_net:
            h = l(h)
            
        results.append(h.detach().cpu().numpy())

In [ ]:
X.shape

In [ ]:
np.vstack(results).shape

In [ ]:
results_pos = np.vstack(results).reshape([512, 512, 512, -1])

In [ ]:
 with open("cache_pos.npy", "wb") as f:
    np.save(f, results_pos, allow_pickle=False)

In [ ]:
 with open("cache_pos_32.npy", "wb") as f:
    np.save(f, results_pos.astype(np.float32), allow_pickle=False)

In [ ]:
 with open("cache_pos_32_p1.npy", "wb") as f:
    np.save(f, results_pos[:, :, :, :4].astype(np.float32), allow_pickle=False)

In [ ]:
 with open("cache_pos_32_p2.npy", "wb") as f:
    np.save(f, results_pos[:, :, :,4:].astype(np.float32), allow_pickle=False)

In [ ]:
img = Image.fromarray(data, 'RGB')
img.save('my.png')

In [ ]:
i = 0
np.interp(results_pos[i, :, :, :4], (results_pos.min(), results_pos.max()), (0, 255)).astype("uint8")

In [ ]:
results_pos_scaled = np.interp(
        results_pos, (results_pos.min(), results_pos.max()), (0, 255)).astype("uint8")

In [ ]:
from PIL import Image
for i in tqdm(range(512)):
    img = Image.fromarray(results_pos_scaled[:, :, i, :3], 'RGB')
    img.save(f'cache_pos_p1_slices_z\cache_pos_32_p1_{i}.png')
    print(i)


In [ ]:
results_dir = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(D.shape[0] / batch))):
        d = torch.tensor(D[k * batch:(k + 1) * batch].astype(np.float32)).to(device)

        x_ = x_[:, 0] * model.resolution * model.resolution + x_[:, 1] * model.resolution + x_[:, 2]
        bids = x_.int()
        result = torch.zeros((x.shape[0], 8)).half()
        for bid in torch.unique(bids):
            pos = x[x_ == bid]
            centers = c[x_ == bid]
            h = model.encoders[bid](pos - centers, bound=model.bound / model.resolution)
            for l in model.sigma_nets[bid]:
                h = l(h)
            result[x_ == bid] = h.detach().cpu()      
        results.append(result.numpy())

In [ ]:
results_dir = []
with torch.cuda.amp.autocast(enabled=True):
    for k in tqdm(range(int(D.shape[0] / batch))):
        h = model.encoder_dir(torch.tensor(D[k * batch:(k + 1) * batch].astype(np.float32)).to(device))
        for l in model.gamma_net:
            h = l(h)
            
        results_dir.append(h.detach().cpu().numpy())

```
tensor([[-0.2913,  0.5830,  0.2158,  ..., -0.2471, -1.0830, -1.0264],
        [-0.2959,  0.5845,  0.2192,  ..., -0.2418, -1.0820, -1.0234],
        [-0.2959,  0.5845,  0.2192,  ..., -0.2418, -1.0820, -1.0234],
        ...,
        [-0.4473,  0.5923,  0.4666,  ...,  0.1472, -1.0771, -0.7944],
        [-0.4473,  0.5918,  0.4678,  ...,  0.1487, -1.0771, -0.7935],
        [-0.4478,  0.5913,  0.4690,  ...,  0.1504, -1.0781, -0.7925]],
       device='cuda:0')
```
```
tensor([[ 0.2699, -0.8343, -0.4807],
        [ 0.2702, -0.8351, -0.4791],
        [ 0.2705, -0.8359, -0.4776],
        ...,
        [ 0.2910, -0.9562,  0.0311],
        [ 0.2910, -0.9562,  0.0334],
        [ 0.2909, -0.9561,  0.0356]], device='cuda:0')
```

In [ ]:
results_dir_ = np.vstack(results_dir).reshape([N, N, -1])

In [ ]:
d = torch.tensor([[ 0.2699, -0.8343, -0.4807],
        [ 0.2702, -0.8351, -0.4791],
        [ 0.2705, -0.8359, -0.4776]])
theta = torch.acos(d[:, 1])
theta = theta / torch.pi
phi = torch.atan2(-d[:, 2], -d[:, 0])
phi = (phi + torch.pi) / (2 * torch.pi)
d = torch.stack((theta , phi), dim=-1)

gamma_index = torch.round(d * (results_dir_.shape[0] - 1)).long()
gamma = results_dir_[gamma_index[:, 0], gamma_index[:, 1]]

In [ ]:
gamma_index

In [ ]:
gamma

In [ ]:
np.nonzero(
    np.logical_and(
        np.logical_and(
            np.isclose(results_dir_[:, :, 0], 0.0781, rtol=1e-2),
            np.isclose(results_dir_[:, :, 1], 0.8110, rtol=1e-2), 
            np.isclose(results_dir_[:, :, 2], -0.9033, rtol=1e-2)
        ),
        np.logical_and(
            np.isclose(results_dir_[:, :, 5], 0.3992, rtol=1e-2),
            np.isclose(results_dir_[:, :, 6], -1.5908, rtol=1e-2), 
            np.isclose(results_dir_[:, :, 7], -0.0550, rtol=1e-2)
        )
    )
)

In [ ]:
d = torch.tensor([[ 0.2699, -0.8343, -0.4807],
        [ 0.2702, -0.8351, -0.4791],
        [ 0.2705, -0.8359, -0.4776]])
torch.atan2(d[:, 0], d[:, 2])

In [ ]:
(1702 / 2048) * 2 * np.pi

In [ ]:
results_dir_[1656][1699]

```
tensor([[ 0.2699, -0.8343, -0.4807],
        [ 0.2702, -0.8351, -0.4791],
        [ 0.2705, -0.8359, -0.4776],
        ...,
        [ 0.2911, -0.9567, -0.0058],
        [ 0.2911, -0.9567, -0.0035],
        [ 0.2911, -0.9567, -0.0012]], device='cuda:0')
```
```
tensor([[ 0.0781,  0.8110, -0.9033,  ...,  0.3992, -1.5908, -0.0550],
        [ 0.0784,  0.8101, -0.9062,  ...,  0.3965, -1.5908, -0.0547],
        [ 0.0789,  0.8096, -0.9082,  ...,  0.3928, -1.5898, -0.0551],
        ...,
        [ 0.0867,  0.6597, -0.8677,  ..., -0.3740, -1.3115,  0.0590],
        [ 0.0874,  0.6587, -0.8662,  ..., -0.3765, -1.3115,  0.0596],
        [ 0.0883,  0.6577, -0.8662,  ..., -0.3799, -1.3115,  0.0610]],
       device='cuda:0', dtype=torch.float16)
```

In [ ]:
D.shape[0]

In [ ]:
np.vstack(results_dir).shape

In [ ]:
results_dir_ = np.vstack(results_dir).reshape([N, N, -1])

In [ ]:
results_dir_.shape

In [ ]:
 with open("cache_dir.npy", "wb") as f:
    np.save(f, results_dir_, allow_pickle=False)

In [ ]:
 with open("cache_dir_32.npy", "wb") as f:
    np.save(f, results_dir_.astype(np.float32), allow_pickle=False)

In [ ]:
 with open("cache_dir_32_p1.npy", "wb") as f:
    np.save(f, results_dir_[:, :, :4].astype(np.float32), allow_pickle=False)

In [ ]:
 with open("cache_dir_32_p2.npy", "wb") as f:
    np.save(f, results_dir_[:, :, 4:].astype(np.float32), allow_pickle=False)

In [ ]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [ ]:
from PIL import Image
import numpy as np

# w, h = 512, 512
data = results_dir_[:, :, :3]
# data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
img = Image.fromarray(np.interp(data, (data.min(), data.max()),(0, 255)).astype("uint8"), 'RGB')
# img.save('my.png')
img.show()

In [ ]:
results_dir_[0][0]